In [ ]:
!pip install yfinance

import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

start_date = "2020-01-01"
end_date = "2023-12-31"
ticker1 = "KO"   
ticker2 = "PEP" 

data1 = yf.download(ticker1, start=start_date, end=end_date)["Adj Close"]
data2 = yf.download(ticker2, start=start_date, end=end_date)["Adj Close"]


df = pd.DataFrame({ticker1: data1, ticker2: data2}).dropna()
df = df.rename(columns={ticker1: "KO", ticker2: "PEP"})
df = df.reset_index()
df.plot(figsize=(12,6), title="Stock Prices: KO vs PEP")
plt.grid()
plt.show()


df["spread"] = df[ticker1] - df[ticker2]
mean = df["spread"].mean()
std = df["spread"].std()
df["z_score"] = (df["spread"] - mean) / std


df["long"] = df["z_score"] < -1
df["short"] = df["z_score"] > 1
df["exit"] = df["z_score"].abs() < 0.5


df["z_score"].plot(figsize=(12,6), title="Z-Score of Spread")
plt.axhline(1, color='r', linestyle='--')
plt.axhline(-1, color='g', linestyle='--')
plt.axhline(0, color='black', linestyle='-')
plt.grid()
plt.show()
 # pnl calculation
position = 0
pnl = []
for i in range(len(df)):
    if df["long"][i]:
        position = 1
    elif df["short"][i]:
        position = -1
    elif df["exit"][i]:
        position = 0
    pnl.append(position * df["spread"].diff().fillna(0)[i])

df["PnL"] = pnl
df["Cum_PnL"] = df["PnL"].cumsum()


df["Cum_PnL"].plot(figsize=(12,6), title="Cumulative PnL")
plt.grid()
plt.show()


: 